# 工具

## 函数作为工具
将智能体要做的事情摘出来，使用函数来实现，避免耦合度高
  
```python
from agents import function_tool, Agent
@function_tool
def function():
    pass
agent = Agent(**kwargs, tools=[])
```

In [ ]:
from agents import Agent, ModelSettings, function_tool, Runner, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"

set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

@function_tool
def get_weather(city: str) -> str:
    """Fetch the weather for a given city.

    Args:
        city: The city to fetch the weather for.
    """
    return f"The weather in {city} is sunny"

agent = Agent(
    name="天气助手",
    instructions="始终用汉赋的形式回答用户",
    model=llm,
    tools=[get_weather],
)

async def main():
    result = await Runner.run(agent, "上海最近适合出去玩吗?")
    print(result.final_output)

await main()


让 agent 作为工具被其他 agent 调用


```python
some_agent = Agent(...)
agent = Agent(tools=[
    some_agent.as_tool(
        tool_name="",
        tool_description="",
    )
    ])
```

In [ ]:
from agents import Agent, Runner, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('DEEPSEEK_API_KEY')
base_url = os.getenv('https://api.deepseek.com')
chat_model = "deepseek/deepseek-chat"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

spanish_agent = Agent(
    name="Spanish agent",
    instructions="You translate the user's message to Spanish",
    model=llm,
)

french_agent = Agent(
    name="French agent",
    instructions="You translate the user's message to French",
    model=llm,
)

orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=(
        "You are a translation agent. You use the tools given to you to translate."
        "If asked for multiple translations, you call the relevant tools."
    ),
    model=llm,
    tools=[
        spanish_agent.as_tool(
            tool_name="translate_to_spanish",
            tool_description="Translate the user's message to Spanish",
        ),
        french_agent.as_tool(
            tool_name="translate_to_french",
            tool_description="Translate the user's message to French",
        ),
    ],
)

def main():
    result = Runner.run_sync(orchestrator_agent, input="Say 'Hello, how are you?' in Spanish.")
    print(result.final_output)

await main()

## agent 作为工具

当 agent 作为 tool 时，可以通过定义函数获得格式化输出

In [ ]:
from agents import Agent, Runner, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
from agents.result import RunResult
from agents.items import ToolCallOutputItem
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('DEEPSEEK_API_KEY')
base_url = os.getenv('https://api.deepseek.com/v1')
chat_model = "deepseek/deepseek-chat"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)


async def extract_json_payload(run_result: RunResult) -> str:
    # Scan the agent’s outputs in reverse order until we find a JSON-like message from a tool call.
    for item in reversed(run_result.new_items):
        if isinstance(item, ToolCallOutputItem) and item.output.strip().startswith("{"):
            return item.output.strip()
    # Fallback to an empty JSON object if nothing was found
    return "{}"

data_agent = Agent(
    name="信息抽取",
    instructions="抽取name和position信息为json格式",
    model=llm,
)

json_tool = data_agent.as_tool(
    tool_name="get_data_json",
    tool_description="Run the data agent and return only its JSON payload",
    custom_output_extractor=extract_json_payload,
)

agent = Agent(
    name="简历优化助手",
    instructions="优化用户的简历信息，特别是把json_tool工具抽取的信息进行充分的扩充",
    model=llm,
    tools=[json_tool],
)

def main():
    result = Runner.run_sync(agent, "我叫李婷，女，现居上海，想要应聘前台岗位。")
    print(result.final_output)

# if __name__ == "__main__":
#     main()

main()

假如 data_agent 处理后输出：`"我从您的信息中提取到以下数据：{'name': '李婷', 'position': '前台'}这些是主要的结构化信息。"`，extract_json_payload函数就从中提取json块

custom_output_extractor() 在子agent返回结果给主agent前被调用，用于规范输出

```mermaid
用户输入 -> agent[简历优化助手] -> 调用工具get_data_json -> data_agent[信息抽取] -> 返回JSON -> 【extract_json_payload在这里执行】-> agent继续处理
```